In [1]:
import pandas as pd
from postal.expand import expand_address
import numpy as np 
from helpers import s3_connection
import helpers 
import matplotlib.pyplot as plt

In [3]:
pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 55.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 74.7 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 KB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 68.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install statsmodels


SyntaxError: invalid syntax (2127436328.py, line 1)

# **DPE ADEME**

### LOGEMENTS EXISTANTS ###

In [2]:
df = pd.read_csv(
    "/Users/noelinecasteil/Documents/statapp/DPE/DPE_ADEME/dpe-v2-logements-existants.csv",
    sep=",",  # Séparateur CSV
    encoding="utf-8",
    low_memory=False)

df['Date_réception_DPE'].count()

4537525

### import données isaure

In [2]:
s3 = s3_connection()


connection successful


In [3]:
path_logements_existants = "clichere/diffusion/DPE/DPE_ADEME/dpe-v2-logements-existants.csv"
df = s3.read_file_from_s3(path_logements_existants)

/home/onyxia/work/STATAPP/helpers.py:35: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_in, usecols=columns_to_select, dtype=dtype_spec, sep=sep)


On filtre pour n'avoir que les données du département 44 en 2022.

In [3]:
df['Date_réception_DPE'] = pd.to_datetime(df['Date_réception_DPE'], errors='coerce')
dfv1 = df[df['Date_réception_DPE'].dt.year == 2022].copy()
dfv2 = dfv1[dfv1['N°_département_(BAN)']=='44'].copy()

In [4]:
dfv2['Date_réception_DPE'].count()

64490

La fonction normalize renvoie des listes de différentes versions d'adresses possibles (ex Chateau-Thabut ; Chateau Thabut ; ChateauThabut)

Avec cette version de la fonction on ne garde que la première composante de la liste. 

In [5]:
def normalize_address(address):
    if pd.isna(address) or address.strip() == '':
        return None  
    try:
        normalized = expand_address(address)  
        return normalized[0] if normalized else None  # Ne garde que la première version
    except Exception as e:
        print(f"Erreur avec l'adresse '{address}': {e}")
        return None

In [6]:
dfv2['Adresse_Normalisee'] = dfv2['Adresse_(BAN)'].apply(normalize_address)

### LOGEMENTS NEUFS ###

In [7]:
df2 = pd.read_csv(
    "/Users/noelinecasteil/Documents/statapp/DPE/DPE_ADEME/dpe-v2-logements-neufs.csv",
    sep=",",  # Séparateur CSV
    encoding="utf-8",
    low_memory=False)

df2['Date_réception_DPE'].count()

537952

### import fichier logement neuf isaure

In [8]:
path_logements_neufs = "clichere/diffusion/DPE/DPE_ADEME/dpe-v2-logements-neufs.csv"
df2 = s3.read_file_from_s3(path_logements_neufs)


/home/onyxia/work/STATAPP/helpers.py:35: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support sep=None with delim_whitespace=False; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_in, usecols=columns_to_select, dtype=dtype_spec, sep=sep)


In [8]:
df2['Date_réception_DPE'] = pd.to_datetime(df2['Date_réception_DPE'], errors='coerce')
df2v1 = df2[df2['Date_réception_DPE'].dt.year == 2022].copy()
df2v2 = df2v1[df2v1['N°_département_(BAN)']=='44'].copy()

In [9]:
df2v2['Date_réception_DPE'].count()

11624

In [10]:
df2v2['Adresse_Normalisee'] = df2v2['Adresse_(BAN)'].apply(normalize_address)

### VALEURS FONCIERES

In [11]:

vf = pd.read_csv(
    "/Users/noelinecasteil/Documents/statapp/ValeursFoncieres/valeursfoncieres-2022.txt",
    sep="|",  
    encoding="utf-8",
    low_memory=False
)

### import données isaure valeurs foncières

In [12]:
path_valeursfoncieres_2022 = "clichere/diffusion/Valeursfoncières/valeursfoncieres-2022.txt"
vf = s3.read_file_from_s3(path_valeursfoncieres_2022, sep="|")

/home/onyxia/work/STATAPP/helpers.py:35: DtypeWarning: Columns (18,23,24,26,28,29,31,33,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_in, usecols=columns_to_select, dtype=dtype_spec, sep=sep)


In [12]:
print(vf[['No voie', 'Type de voie', 'Voie', 'Code postal', 'Commune']].dtypes)


No voie         float64
Type de voie     object
Voie             object
Code postal     float64
Commune          object
dtype: object


In [13]:
# Convertir explicitement toutes les colonnes en chaînes
vf['Adresse'] = vf['No voie'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
                vf['Type de voie'].fillna('').astype(str) + " " + \
                vf['Voie'].fillna('').astype(str) + ", " + \
                vf['Code postal'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
                vf['Commune'].fillna('').astype(str)

vf['Adresse'] = vf['Adresse'].str.strip().replace(r'^\s*$', None, regex=True)  # Supprime les adresses vides


In [14]:
vf44 = vf[vf['Code postal'].notna()]
vf44 = vf44[vf44['Code postal'].astype(str).str.startswith('44')].copy()

In [15]:
vf44['Adresse'] = vf44['Adresse'].str.strip().str.replace(r'\s+', ' ', regex=True)

In [16]:
vf44['Adresse_Normalisee'] = vf44['Adresse'].apply(normalize_address)

In [17]:
print(len(vf44['Adresse_Normalisee']))

99874


## **MATCHING ETUDE**

In [18]:
def unique(df):
    adresse_counts = df['Adresse_Normalisee'].value_counts()
    nb_adresses_uniques = (adresse_counts == 1).sum()
    return nb_adresses_uniques

In [19]:
unique(dfv2)

22972

In [20]:
unique(vf44)

16910

In [21]:
print(f"Nombre d'adresses dupliquées dans dfv2 (logements existants) : {dfv2 ['Adresse_Normalisee'].value_counts().ge(2).sum()}")
print(f"Nombre d'adresses dupliquées dans df2v2 (logements neufs) : {df2v2 ['Adresse_Normalisee'].value_counts().ge(2).sum()}")
print(f"Nombre d'adresses dupliquées dans vf44 (valeurs foncières) : {vf44 ['Adresse_Normalisee'].value_counts().ge(2).sum()}")

Nombre d'adresses dupliquées dans dfv2 (logements existants) : 7513
Nombre d'adresses dupliquées dans df2v2 (logements neufs) : 707
Nombre d'adresses dupliquées dans vf44 (valeurs foncières) : 20674


TRAVAIL SUR LES DOUBLONS

DOUBLONS DANS DOC LOGEMENTS EXISTANTS

In [22]:
def calculate_duplicate_percentage(df, address_column):
    """
    On calcule le pourcentage d'adresses en doublon + on en fait une liste
    """
    total_count = len(df)
    duplicate_counts = df[address_column].value_counts()
    duplicate_addresses = duplicate_counts[duplicate_counts > 1].index.tolist()
    duplicate_count = len(duplicate_addresses)
    duplicate_percentage = (duplicate_count / total_count) * 100 if total_count > 0 else 0
    
    return duplicate_percentage, duplicate_addresses

In [23]:
# On applique la fonction aux logements existants de dfv2
duplicate_percentage, duplicate_addresses = calculate_duplicate_percentage(dfv2, 'Adresse_Normalisee')
print(f"Pourcentage d'adresses en doublon : {duplicate_percentage:.2f}%")
print("Liste des adresses en doublon :")
print(duplicate_addresses[:10])
print(len(duplicate_addresses))

Pourcentage d'adresses en doublon : 11.65%
Liste des adresses en doublon :
['1 rue de cahors 44800 saint-herblain', '5 avenue robert chasteland 44700 orvault', '32 route de la joneliere 44300 nantes', '24 rue blaise pascal 44300 nantes', 'route de saint joseph 44300 nantes', '4 avenue des jades 44300 nantes', 'rue de la coran 44400 reze', '8 place francois ii 44200 nantes', '129 rue de la mirette 44400 reze', '5 rue de biarritz 44200 nantes']
7513


DOUBLONS DANS VF44

In [24]:
# On applique la fonction aux logements existants de vf44
duplicate_percentage2, duplicate_addresses2 = calculate_duplicate_percentage(vf44, 'Adresse_Normalisee')
print(f"Pourcentage d'adresses en doublon : {duplicate_percentage2:.2f}%")
liste2 = duplicate_addresses2
print("Liste des adresses en doublon :")
print(liste2[:10])
print(len(liste2))

Pourcentage d'adresses en doublon : 20.70%
Liste des adresses en doublon :
['13 rue de saint servan 44800 st-herblain', 'rue de la jaunaie 44230 saint sebastien sur loire', 'rue joshua slocum 44210 pornic', '51 rue hector berlioz 44300 nantes', 'zone industrielle de brais 44600 saint-nazaire', '43 boulevard des batignolles 44300 nantes', '66 rue de nantes 44830 bouaye', 'favet 44650 corcoue-sur-logne', '95 avenue de la patouillerie 44700 orvault', 'le butay 44140 montbert']
20674


ON CHERCHE LES ADRESSES A MATCH PARMI CES DOUBLONS

In [25]:
set_adresses = set(duplicate_addresses)
set_adresses2 = set(duplicate_addresses2)
# Adresses en commun
adresses_match = set_adresses.intersection(set_adresses2)
print(len(adresses_match))

1881


Donc sur toutes les adresses en doublons il y aurait seulement 1881 potentiellement à match. 

On calcule la perte potentielle si on venait à enlever tous les doublons et ne garder que les adresses uniques.

In [26]:
longueur = len(vf44)+len(dfv2)
print(longueur)
match_adresses_uniques = unique(dfv2)+unique(vf44)
print(match_adresses_uniques)
print(match_adresses_uniques*100/longueur)

164364
39882
24.2644374680587


On perdrait donc environ 24% de l'information.

CREATION DUNE LISTE AVEC LES ADRESSES UNIQUES ET EN DOUBLON POUR COMPTER NOMBRE DE MATCHING A EFFECTUER

In [27]:
adresse_counts = dfv2['Adresse_Normalisee'].value_counts()

# Séparer les adresses uniques et celles en doublon (qu'on ne garde qu'une seule fois)
adresses_uniques = adresse_counts[adresse_counts == 1].index.tolist()
adresses_doublons = adresse_counts[adresse_counts > 1].index.tolist()

# Fusionner les deux listes
adresses_finales = adresses_uniques + adresses_doublons

# Affichage du résultat
print("Liste des adresses uniques + adresses en doublon (mais une seule fois) :")
print(adresses_finales[:10])
print(len(adresses_finales))

Liste des adresses uniques + adresses en doublon (mais une seule fois) :
['7 rue de courtigon 44210 pornic', '46 rue de la bourdonnais 44100 nantes', '35 rue de la chicotiere 44800 saint-herblain', '23 avenue centre henri de cosse brissac 44540 vallons-de-lerdre', '51 rue du grand puits 44450 divatte-sur-loire', '10 rue georges clemenceau 44340 bouguenais', '48ter rue des primeveres 44220 coueron', '29 avenue des noes 44380 pornichet', '18 square des tilleuls 44522 mesanger', '23 rue herve bazin 44130 blain']
30485


In [28]:
adresse_counts = vf44['Adresse_Normalisee'].dropna().value_counts()


adresses_uniques = list(adresse_counts[adresse_counts == 1].index)
adresses_doublons = list(adresse_counts[adresse_counts > 1].index)

adresses_finales2 = adresses_uniques + adresses_doublons

print(f"Taille finale de la liste : {len(adresses_finales2)}")
print("Liste des adresses uniques + adresses en doublon (mais une seule fois) :")
print(adresses_finales2[:10])


Taille finale de la liste : 37584
Liste des adresses uniques + adresses en doublon (mais une seule fois) :
['l ouche du puits fresnay 44580 villeneuve-en-retz', 'le landier du ruaud 44410 asserac', '17 rue du puy civaux 44580 villeneuve-en-retz', 'les malabris 44130 notre dame des landes', '5362 le perron 44160 besne', '83 l annerie 44190 getigne', 'les landes de la piolais 44160 crossac', '3 rue du taillis 44140 montbert', '2 impasse des papillons 44450 saint-julien-de-concelles', '1 rue des chaloires 44760 la bernerie enceinte retz']


In [29]:
# Conversion des listes en ensemble
set_adresses_finales = set(adresses_finales)
set_adresses_finales2 = set(adresses_finales2)

# Adresses en commun
adresses_communes = set_adresses_finales.intersection(set_adresses_finales2)

print(f"Nombre d'adresses en commun : {len(adresses_communes)}")

# Exemple d'adresses en commun
print("Exemples d'adresses en commun :", list(adresses_communes)[:10])


Nombre d'adresses en commun : 7120
Exemples d'adresses en commun : ['53 route de la barre 44830 bouaye', '25 rue des jaunins 44580 villeneuve-en-retz', '18 rue du luxembourg 44980 sainte-luce-sur-loire', '9 rue georges clemenceau 44470 carquefou', '7 place des tamaris 44190 clisson', '18 square des tilleuls 44522 mesanger', '6 rue de la pommeraie 44530 dreffeac', '10 place ladmirault 44770 la plaine-sur-mer', 'route de saint herblain 44100 nantes', '22 avenue delaware la close 44300 nantes']


MEILLEURE VISION DES DOUBLONS EN CSV

Attention à ne pas push sur git le fichier csv généré (il est trop lourd)

In [31]:
def export_duplicate_addresses(dfv2, vf44, output_file):
    # Identifier les adresses en doublon dans vf44
    vf44_dupes = vf44[vf44['Adresse_Normalisee'].duplicated(keep=False)]
    
    # Identifier les adresses en doublon dans dfv2
    dfv2_dupes = dfv2[dfv2['Adresse_Normalisee'].duplicated(keep=False)]
    
    # Trouver les adresses en commun entre les deux jeux de données
    common_addresses = set(vf44_dupes['Adresse_Normalisee']).intersection(set(dfv2_dupes['Adresse_Normalisee']))
    
    # Filtrer les données pour ne conserver que celles ayant une adresse en commun
    vf44_common = vf44[vf44['Adresse_Normalisee'].isin(common_addresses)].copy()
    dfv2_common = dfv2[dfv2['Adresse_Normalisee'].isin(common_addresses)].copy()
    
    # Ajouter une colonne Source pour identifier l'origine des données
    vf44_common['Source'] = 'vf44'
    dfv2_common['Source'] = 'dfv2'
    
    # Harmoniser les colonnes des deux dataframes
    all_columns = list(set(dfv2_common.columns).union(set(vf44_common.columns)))
    vf44_common = vf44_common.reindex(columns=all_columns)
    dfv2_common = dfv2_common.reindex(columns=all_columns)
    
    # Concaténer les deux jeux de données sans perte d'information
    all_common_dupes = pd.concat([vf44_common, dfv2_common], ignore_index=True)
    
    # Trier par adresse normalisée pour regrouper les doublons
    all_common_dupes = all_common_dupes.sort_values(by=['Adresse_Normalisee'])
    
    # Exporter en CSV
    all_common_dupes.to_csv(output_file, index=False, encoding='utf-8')
    print(f"Fichier CSV exporté : {output_file}")

export_duplicate_addresses(dfv2, vf44, "adresses_doublon_communes.csv")

/tmp/ipykernel_56901/2984065862.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_common_dupes = pd.concat([vf44_common, dfv2_common], ignore_index=True)


Fichier CSV exporté : adresses_doublon_communes.csv


On remarque un problème avec les dépendances qui ont les mêmes infos que les appartements dans lesquels elles sont situées. On veut voir quelle quantité elles représentent.

In [30]:
# Filtre
dependance_count = vf44[vf44['Type local'] == 'Dépendance'].shape[0]

print(f"Nombre de lignes avec le type de local 'Dépendance' dans vf44 : {dependance_count}")
# Nombre total de lignes
print(len(vf44['Type local']))

Nombre de lignes avec le type de local 'Dépendance' dans vf44 : 26009
99874


In [38]:
df_doublons = pd.read_csv("adresses_doublon_communes.csv", encoding="utf-8")
df_logements = pd.read_csv("logements_existants_adresse_commune.csv", encoding="utf-8")

# Afficher les premières lignes des fichiers pour vérifier
display(df_doublons)  
display(df_logements.head())

/tmp/ipykernel_56901/866023129.py:1: DtypeWarning: Columns (4,16,40,52,54,73) have mixed types. Specify dtype option on import or set low_memory=False.
  df_doublons = pd.read_csv("adresses_doublon_communes.csv", encoding="utf-8")


,Nom__commune_(BAN),Adresse_brute,Identifiant de document,3 Articles CGI,B/T/Q,Nombre de lots,N°_département_(BAN),No Volume,Modèle_DPE,Nature culture,...,No voie,Date mutation,Adresse,Adresse_Normalisee,Surface Carrez du 4eme lot,Surface terrain,No disposition,Nom__commune_(Brut),Etiquette_GES,Nom__rue_(BAN)
0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,1.0,20/06/2022,"1 ALL CHARLES BAUDELAIRE, 44400 REZE",1 allee charles baudelaire 44400 reze,NaN,NaN,1.0,NaN,NaN,NaN
1,Rezé,Charles Baudelaire,NaN,NaN,NaN,NaN,44.0,NaN,DPE 3CL 2021 méthode logement,NaN,...,NaN,NaN,NaN,1 allee charles baudelaire 44400 reze,NaN,NaN,NaN,REZÉ,C,Allée Charles Baudelaire
2,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,1.0,03/10/2022,"1 ALL CHARLES BAUDELAIRE, 44400 REZE",1 allee charles baudelaire 44400 reze,NaN,NaN,1.0,NaN,NaN,NaN
3,Rezé,"1, Allée Charles Baudelaire",NaN,NaN,NaN,NaN,44.0,NaN,DPE 3CL 2021 méthode logement,NaN,...,NaN,NaN,NaN,1 allee charles baudelaire 44400 reze,NaN,NaN,NaN,REZE,E,Allée Charles Baudelaire
4,Rezé,1 ALLÉE CHARLES BAUDELAIRE,NaN,NaN,NaN,NaN,44.0,NaN,DPE 3CL 2021 méthode logement,NaN,...,NaN,NaN,NaN,1 allee charles baudelaire 44400 reze,NaN,NaN,NaN,REZE,E,Allée Charles Baudelaire
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17256,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,J,...,NaN,22/12/2022,"TREVECART, 44500 LA BAULE-ESCOUBLAC",trevecart 44500 la baule-escoublac,NaN,2660.0,1.0,NaN,NaN,NaN
17257,La Baule-Escoublac,95 chemin de Trévécart,NaN,NaN,NaN,NaN,44.0,NaN,DPE 3CL 2021 méthode logement,NaN,...,NaN,NaN,NaN,trevecart 44500 la baule-escoublac,NaN,NaN,NaN,La Baule-Escoublac,A,95 chemin de Trévécart
17258,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,07/02/2022,"TREVECART, 44500 LA BAULE-ESCOUBLAC",trevecart 44500 la baule-escoublac,NaN,NaN,1.0,NaN,NaN,NaN
17259,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,18/01/2022,"TREVECART, 44500 LA BAULE-ESCOUBLAC",trevecart 44500 la baule-escoublac,NaN,NaN,1.0,NaN,NaN,NaN


,Date_réception_DPE,Date_établissement_DPE,Modèle_DPE,Date_fin_validité_DPE,Version_DPE,Méthode_application_DPE,Etiquette_DPE,Etiquette_GES,Année_construction,Type_bâtiment,...,N°_étage_appartement,Nom_résidence,Cage_d'escalier,Complément_d'adresse_logement,Statut_géocodage,Nom__commune_(Brut),N°_département_(BAN),N°_région_(BAN),Complément_d'adresse_bâtiment,Adresse_Normalisee
0,2022-02-01,2022-02-01,DPE 3CL 2021 méthode logement,2032-01-31,2.0,dpe appartement individuel,E,B,1961.0,appartement,...,NaN,NaN,NaN,NaN,adresse non géocodée ban car aucune correspond...,NANTES,44,52.0,NaN,20 rue du gue robert 44000 nantes
1,2022-02-01,2022-02-01,DPE 3CL 2021 méthode logement,2032-01-31,2.0,dpe appartement individuel,D,B,1961.0,appartement,...,NaN,NaN,NaN,NaN,adresse non géocodée ban car aucune correspond...,NANTES,44,52.0,NaN,20 rue du gue robert 44000 nantes
2,2022-02-01,2022-02-01,DPE 3CL 2021 méthode logement,2032-01-31,2.0,dpe appartement individuel,E,B,1961.0,appartement,...,NaN,NaN,NaN,NaN,adresse non géocodée ban car aucune correspond...,NANTES,44,52.0,NaN,20 rue du gue robert 44000 nantes
3,2022-10-05,2022-10-05,DPE 3CL 2021 méthode logement,2032-10-04,2.1,dpe appartement individuel,D,B,1948.0,appartement,...,NaN,étage 1 porte à Droite,NaN,1--1-LogZone,adresse non géocodée ban car aucune correspond...,NANTES,44,52.0,NaN,20 rue du gue robert 44000 nantes
4,2022-02-01,2022-02-01,DPE 3CL 2021 méthode logement,2032-01-31,2.0,dpe maison individuelle,F,C,1961.0,maison,...,NaN,NaN,NaN,NaN,adresse non géocodée ban car aucune correspond...,NANTES,44,52.0,NaN,20 rue du gue robert 44000 nantes


In [39]:
df_doublons[df_doublons.duplicated(subset=["Adresse_Normalisee"], keep=False)]


,Nom__commune_(BAN),Adresse_brute,Identifiant de document,3 Articles CGI,B/T/Q,Nombre de lots,N°_département_(BAN),No Volume,Modèle_DPE,Nature culture,...,No voie,Date mutation,Adresse,Adresse_Normalisee,Surface Carrez du 4eme lot,Surface terrain,No disposition,Nom__commune_(Brut),Etiquette_GES,Nom__rue_(BAN)
0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,1.0,20/06/2022,"1 ALL CHARLES BAUDELAIRE, 44400 REZE",1 allee charles baudelaire 44400 reze,NaN,NaN,1.0,NaN,NaN,NaN
1,Rezé,Charles Baudelaire,NaN,NaN,NaN,NaN,44.0,NaN,DPE 3CL 2021 méthode logement,NaN,...,NaN,NaN,NaN,1 allee charles baudelaire 44400 reze,NaN,NaN,NaN,REZÉ,C,Allée Charles Baudelaire
2,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,1.0,03/10/2022,"1 ALL CHARLES BAUDELAIRE, 44400 REZE",1 allee charles baudelaire 44400 reze,NaN,NaN,1.0,NaN,NaN,NaN
3,Rezé,"1, Allée Charles Baudelaire",NaN,NaN,NaN,NaN,44.0,NaN,DPE 3CL 2021 méthode logement,NaN,...,NaN,NaN,NaN,1 allee charles baudelaire 44400 reze,NaN,NaN,NaN,REZE,E,Allée Charles Baudelaire
4,Rezé,1 ALLÉE CHARLES BAUDELAIRE,NaN,NaN,NaN,NaN,44.0,NaN,DPE 3CL 2021 méthode logement,NaN,...,NaN,NaN,NaN,1 allee charles baudelaire 44400 reze,NaN,NaN,NaN,REZE,E,Allée Charles Baudelaire
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17256,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,J,...,NaN,22/12/2022,"TREVECART, 44500 LA BAULE-ESCOUBLAC",trevecart 44500 la baule-escoublac,NaN,2660.0,1.0,NaN,NaN,NaN
17257,La Baule-Escoublac,95 chemin de Trévécart,NaN,NaN,NaN,NaN,44.0,NaN,DPE 3CL 2021 méthode logement,NaN,...,NaN,NaN,NaN,trevecart 44500 la baule-escoublac,NaN,NaN,NaN,La Baule-Escoublac,A,95 chemin de Trévécart
17258,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,07/02/2022,"TREVECART, 44500 LA BAULE-ESCOUBLAC",trevecart 44500 la baule-escoublac,NaN,NaN,1.0,NaN,NaN,NaN
17259,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,18/01/2022,"TREVECART, 44500 LA BAULE-ESCOUBLAC",trevecart 44500 la baule-escoublac,NaN,NaN,1.0,NaN,NaN,NaN


PREMIER TEST MATCHING

In [31]:
print(dfv2.columns)

Index(['Date_réception_DPE', 'Date_établissement_DPE', 'Modèle_DPE',
       'Date_fin_validité_DPE', 'Version_DPE', 'Méthode_application_DPE',
       'Etiquette_DPE', 'Etiquette_GES', 'Année_construction', 'Type_bâtiment',
       'Période_construction', 'Surface_habitable_logement', 'Adresse_brute',
       'Nom__commune_(BAN)', 'Code_INSEE_(BAN)', 'N°_voie_(BAN)',
       'Identifiant__BAN', 'Adresse_(BAN)', 'Code_postal_(BAN)', 'Score_BAN',
       'Nom__rue_(BAN)', 'Coordonnée_cartographique_X_(BAN)',
       'Coordonnée_cartographique_Y_(BAN)', 'Code_postal_(brut)',
       'N°_étage_appartement', 'Nom_résidence', 'Cage_d'escalier',
       'Complément_d'adresse_logement', 'Statut_géocodage',
       'Nom__commune_(Brut)', 'N°_département_(BAN)', 'N°_région_(BAN)',
       'Complément_d'adresse_bâtiment', 'Adresse_Normalisee'],
      dtype='object')


In [32]:
print(vf44.columns)

Index(['Identifiant de document', 'Reference document', '1 Articles CGI',
       '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI',
       'No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune', 'Prefixe de section',
       'Section', 'No plan', 'No Volume', '1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot',
       'Nombre de lots', 'Code type local', 'Type local', 'Identifiant local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain', 'Adresse',
       'Adresse_Normalisee'],
      dtype='object')


TEST

Le code prend 2min environ à tourner

Premier test de matching avec seulement la règle des 5%

In [ ]:
def test_match(vf44,dfv2):
    adresse_counts = vf44['Adresse_Normalisee'].dropna().value_counts()
    unique1 = list(adresse_counts[adresse_counts == 1].index)
    doublons1 = list(adresse_counts[adresse_counts > 1].index)
    final = unique1 + doublons1
    set_final = set(final)

    adresse_counts2 = dfv2['Adresse_Normalisee'].dropna().value_counts()
    unique2 = list(adresse_counts2[adresse_counts2 == 1].index)
    doublons2 = list(adresse_counts2[adresse_counts2 > 1].index)
    final2 = unique2 + doublons2
    set_final2 = set(final2)

    commun = set_final.intersection(set_final2)

    vf44['Surface Carrez du 1er lot'] = pd.to_numeric(
        vf44['Surface Carrez du 1er lot'].astype(str).str.replace(',', '.'), errors='coerce'
    )
    dfv2['Surface_habitable_logement'] = pd.to_numeric(
        dfv2['Surface_habitable_logement'].astype(str).str.replace(',', '.'), errors='coerce'
    )

    merged = []

    for adresse in commun:
        dfv2sub = dfv2[dfv2['Adresse_Normalisee'] == adresse]
        vf44sub = vf44[vf44['Adresse_Normalisee'] == adresse]

        if len(dfv2sub)==1 and len(vf44sub)==1:
            merged.append({**dfv2sub.iloc[0].to_dict(), **vf44sub.iloc[0].to_dict()})
        else :
             #boucle sur dfv2sub
            for _, row2 in dfv2sub.iterrows():
                matched = False
                for _, row1 in vf44sub.iterrows():
                    surface1 = row1['Surface Carrez du 1er lot']
                    surface2 = row2['Surface_habitable_logement']
                    
                    #écart de 5% de surface
                    if abs(surface1 - surface2) / max(surface1, surface2) < 0.05:
                        merged.append({**row2.to_dict(), **row1.to_dict()})
                        matched = True
                        break
                
                if not matched:
                    merged.append(row2.to_dict())
            
            #ajouter éléments si pas ajoutés
            for _, row1 in vf44sub.iterrows():
                if not any(row1['Adresse_Normalisee'] == m['Adresse_Normalisee'] for m in merged):
                    merged.append(row1.to_dict())
        
    dfv2_uniques = dfv2[~dfv2['Adresse_Normalisee'].isin(commun)]
    print(len(dfv2_uniques))
    vf44_uniques = vf44[~vf44['Adresse_Normalisee'].isin(commun)]
    print(len(vf44_uniques))

    df = pd.DataFrame(merged)
    df = pd.concat([df, dfv2_uniques, vf44_uniques], ignore_index=True)

    return df

test = test_match(vf44,dfv2)
    

50675
78240


In [41]:
print(len(test))

142730


In [42]:
print(len(dfv2))
print(len(vf44))

64490
99874


Deuxième test de matching

In [45]:
def test_match2(vf44, dfv2):
    adresse_counts = vf44['Adresse_Normalisee'].dropna().value_counts()
    unique1 = list(adresse_counts[adresse_counts == 1].index)
    doublons1 = list(adresse_counts[adresse_counts > 1].index)
    final = unique1 + doublons1
    set_final = set(final)

    adresse_counts2 = dfv2['Adresse_Normalisee'].dropna().value_counts()
    unique2 = list(adresse_counts2[adresse_counts2 == 1].index)
    doublons2 = list(adresse_counts2[adresse_counts2 > 1].index)
    final2 = unique2 + doublons2
    set_final2 = set(final2)

    commun = set_final.intersection(set_final2)

    vf44['Surface Carrez du 1er lot'] = pd.to_numeric(
        vf44['Surface Carrez du 1er lot'].astype(str).str.replace(',', '.'), errors='coerce'
    )
    dfv2['Surface_habitable_logement'] = pd.to_numeric(
        dfv2['Surface_habitable_logement'].astype(str).str.replace(',', '.'), errors='coerce'
    )

    merged = []

    for adresse in commun:
        dfv2sub = dfv2[dfv2['Adresse_Normalisee'] == adresse]
        vf44sub = vf44[vf44['Adresse_Normalisee'] == adresse]

        for _, row2 in dfv2sub.iterrows():
            best_match = None
            best_value = -1

            for _, row1 in vf44sub.iterrows():
                surface1 = row1['Surface Carrez du 1er lot']
                surface2 = row2['Surface_habitable_logement']

                #si surfaces identiques on match directement
                if surface1 == surface2:
                    best_match = row1
                    break
                
                #écart inférieur à 5%
                if abs(surface1 - surface2) / max(surface1, surface2) < 0.05:
                    valeur_fonciere = pd.to_numeric(str(row1.get('Valeur fonciere', 0)).replace(',', '.'), errors='coerce')
                    if valeur_fonciere > best_value:
                        best_value = valeur_fonciere
                        best_match = row1

            if best_match is not None:
                merged.append({**row2.to_dict(), **best_match.to_dict()})


    df = pd.DataFrame(merged)
    return df

test2 = test_match2(vf44, dfv2)


In [46]:
test2.to_csv("test2.csv", index=False, encoding='utf-8')

In [47]:
print(len(test2))

3201


In [43]:
#adresses en doublons
dfv2_dupes = dfv2[dfv2['Adresse_Normalisee'].duplicated(keep=False)]
vf44_dupes = vf44[vf44['Adresse_Normalisee'].duplicated(keep=False)]

#adresses communes aux deux fichiers
common_addresses = set(dfv2_dupes['Adresse_Normalisee']).intersection(set(vf44_dupes['Adresse_Normalisee']))

#on sélectionne une adresse commune
if common_addresses:
    adresse_commune = list(common_addresses)[0]  
    print(f"Adresse commune trouvée : {adresse_commune}")
    
    #lignes correspondantes
    dfv2_common = dfv2[dfv2['Adresse_Normalisee'] == adresse_commune]
    vf44_common = vf44[vf44['Adresse_Normalisee'] == adresse_commune]
    
    #transfo csv
    dfv2_common.to_csv("logements_existants_adresse_commune.csv", index=False, encoding='utf-8')
    vf44_common.to_csv("valeurs_foncieres_adresse_commune.csv", index=False, encoding='utf-8')
    
    print("Ok.")
else:
    print("Bug")



Adresse commune trouvée : 15 rue kervegan 44000 nantes
Ok.


In [44]:
# Vérifier combien d'adresses normalisées sont en commun dans test
nb_match = len(set(test["Adresse_Normalisee"]) & set(vf44["Adresse_Normalisee"]) & set(dfv2["Adresse_Normalisee"]))
print(f"Nombre d'adresses effectivement matchées : {nb_match}")


Nombre d'adresses effectivement matchées : 7120


In [45]:
duplicated_test = test["Adresse_Normalisee"].value_counts()
duplicated_test = duplicated_test[duplicated_test > 1]
print(f"Nombre d'adresses apparaissant plusieurs fois après fusion : {len(duplicated_test)}")


Nombre d'adresses apparaissant plusieurs fois après fusion : 23418


## on regarde les adresses qui reviennent le plus souvent


In [46]:
duplicated_test[duplicated_test > 1].head(10)


Adresse_Normalisee
1 rue de cahors 44800 saint-herblain        774
5 avenue robert chasteland 44700 orvault    377
32 route de la joneliere 44300 nantes       264
24 rue blaise pascal 44300 nantes           252
13 rue de saint servan 44800 st-herblain    214
route de saint joseph 44300 nantes          199
4 avenue des jades 44300 nantes             167
rue de la coran 44400 reze                  162
8 place francois ii 44200 nantes            160
129 rue de la mirette 44400 reze            158
Name: count, dtype: int64